In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import sys
from pathlib import Path
sys.path.insert(1, str(Path(os.getcwd()).parent / "src"))

In [ ]:
import numpy as np
import data_loader
import config
import pmcx
from matplotlib import pyplot as plt
from mcs_function import SimulationAttenuation
from plotting import *
import time

In [ ]:
pmcx.gpuinfo()

In [ ]:
config.gpuid

In [ ]:
config.mcs_func_path

In [ ]:
mu_s_bounds = np.concatenate((
    data_loader.DataLoader.mu_s_red_func_gray_matter(np.array([450, 1000])) / (1-0.85),
    data_loader.DataLoader.mu_s_red_func_blood_vessel(np.array([450, 1000])) / (1-0.935)
))
print(mu_s_bounds)

In [ ]:
mu_s_vals = np.linspace(10, 500, 30)
print(mu_s_vals)

In [ ]:
g_combined = 0.8675
refractive_index_combined = 1.38

In [ ]:
# very deep tissue necessary, since no absorption allows photons to go deep into tissue
# for faster and statiscally better results, increasy voxel size
vol = np.ones((50, 50, 1000))
prop = np.array([[0, 0, 1, 1], [0, 0, g_combined, refractive_index_combined]])
unitinmm = 10

cfg = {
    "nphoton": 5e4, # keep nphotons relatively low, to limit memory of stored data
    "maxdetphoton": 5e4,
    "unitinmm": unitinmm,
    "vol": vol,
    "tstart":0,
    "tend":5e-7,
    "tstep":5e-7,
    "autopilot": 1,
    "gpuid": config.gpuid,
    "prop":prop,
    "bc": "ccrcca001000", # mark z=0 plane as detector
    "srcdir": [0,0,1],
    "srctype": "planar",
    "srcpos": [0, 0, 0],
    "srcparam1": [vol.shape[0], 0, 0, 0], # 3D position of vertex, fourth coordinate is irrelevant
    "srcparam2": [0, vol.shape[1], 0, 0],
    "issrcfrom0": 1,
    "savedetflag": "dps", # detector id, path length, scatter count
    "flog": config.mcs_func_path / "log.txt",
}

In [ ]:
function_data = []
function_data.append(cfg["nphoton"])
function_data.append(cfg["prop"][1, 2])
function_data.append(mu_s_vals)

In [ ]:
for i, mu_s in enumerate(mu_s_vals):
    cfg["prop"][1, 1] = mu_s / 10 # mu_s_vals are stored in cm^-1
    cfg["prop"][1, 0] = 0
    cfg["prop"][1, 2] = g_combined
    cfg["prop"][1, 3] = refractive_index_combined
    print(f"Iteration {i}/{len(mu_s_vals)}")
    res = pmcx.mcxlab(cfg)
    cur_photon_data = np.row_stack(
        (
            res["detp"]["ppath"][:, 0] * cfg["unitinmm"] / 10, # store pathlength in cm. pathlengths are stored in "unitinmm"
            res["detp"]["nscat"][:, 0]
        )
    )
    print(f"Mean path length: {pmcx.utils.meanpath(res['detp'])}")
    function_data.append(cur_photon_data)

np.savez(config.mcs_func_path, *function_data)

Load Data and plot for some mu_a values

In [ ]:
function_data_loaded = np.load(config.mcs_func_path)
nphoton = function_data_loaded["arr_0"]
g = function_data_loaded["arr_1"]
mu_s_vals = function_data_loaded["arr_2"]

In [ ]:
mu_a_bounds = np.concatenate((
    data_loader.DataLoader.mu_a_func_gray_matter(np.array([450, 999])),
    data_loader.DataLoader.mu_a_func_blood_vessel(np.array([450, 999])) 
))
print(mu_a_bounds)
mu_a_samples = np.linspace(np.min(mu_a_bounds) * 0.75, np.max(mu_a_bounds) * 1.25, 5)

In [ ]:
for mu_a in mu_a_samples:
    attenuation_per_mu_a = []
    for i, mu_s in enumerate(mu_s_vals):
        photon_data = function_data_loaded[f"arr_{i + 3}"]
        photon_ppath = photon_data[0, :]
        #photon_nscat = photon_data[1, :]
        photon_weights = np.exp(-mu_a * photon_ppath)
        attenuation = -np.log(np.sum(photon_weights) / nphoton)
        attenuation_per_mu_a.append(attenuation)

    plt.scatter(mu_s_vals, attenuation_per_mu_a, label=f"mu_a={mu_a}")
plt.legend()
plt.xlabel("mu_s")
plt.ylabel("Attenuation")


Same plot but with randomly sampled, then interpolated values for mu_s...

In [ ]:
mcs_obj = SimulationAttenuation(config.mcs_func_path)

In [ ]:
for mu_a in mu_a_samples:
    attenuation_per_mu_a = []
    for i, mu_s in enumerate(mu_s_vals):
        photon_data = function_data_loaded[f"arr_{i + 3}"]
        photon_ppath = photon_data[0, :]
        #photon_nscat = photon_data[1, :]
        photon_weights = np.exp(-mu_a * photon_ppath)
        attenuation = -np.log(np.sum(photon_weights) / nphoton)
        attenuation_per_mu_a.append(attenuation)
    
    p = plt.scatter(mu_s_vals, attenuation_per_mu_a, label=f"mu_a={mu_a}")

    mu_s_vals_sampled = np.random.rand(10) * (mu_s_vals[-1] - mu_s_vals[0]) * 1.25 + mu_s_vals[0]
    attenuation_interpolated =  mcs_obj.A(mu_a, mu_s_vals_sampled)
    plt.scatter(mu_s_vals_sampled, attenuation_interpolated, marker="x", color=p.get_facecolor()[0])

    
plt.legend()
plt.xlabel("mu_s")
plt.ylabel("Attenuation")

Sampe graph, but compare with attenuation based on perturbation Monte Carlo method, with mean mu_a, mu_s values as baseline.
This barely works because of overflows.

In [ ]:
mu_a_baseline = mu_a_samples[len(mu_a_samples)//2]
mu_s_baseline = mu_s_vals[-1]
photon_ppath_baseline, photon_nscat_baseline = function_data_loaded[f"arr_{2 + len(mu_s_vals) - 1}"][:, :]
weights_baseline = np.exp(-mu_a_baseline * photon_ppath_baseline)

In [ ]:
np.max(photon_ppath_baseline)

In [ ]:
for mu_a in mu_a_samples:
    attenuation_per_mu_a = []
    attenuation_per_mu_a_perturbed = []
    for i, mu_s in enumerate(mu_s_vals):
        photon_data = function_data_loaded[f"arr_{i + 3}"]
        photon_ppath = photon_data[0, :]
        #photon_nscat = photon_data[1, :]
        photon_weights = np.exp(-mu_a * photon_ppath)
        attenuation = -np.log(np.sum(photon_weights) / nphoton)
        attenuation_per_mu_a.append(attenuation)

    p = plt.scatter(mu_s_vals, attenuation_per_mu_a, label=f"mu_a={mu_a}")

    mu_s_vals_sampled = np.random.rand(10) * (mu_s_vals[-1] - mu_s_vals[0]) + mu_s_vals[0]
    photon_weights_perturbed = weights_baseline[None, :] * np.power((mu_s_vals_sampled/mu_s_baseline)[:, None], photon_nscat_baseline[None, :])
    photon_weights_perturbed *= np.exp(-(mu_s_vals_sampled - mu_s_baseline)[:, None] * photon_ppath_baseline[None, :])
    photon_weights_perturbed *= np.exp(-(mu_a - mu_a_baseline) * photon_ppath_baseline[None, :])
    attenuation_perturbed = -np.log(np.sum(photon_weights_perturbed, axis=-1) / nphoton)

    plt.scatter(mu_s_vals_sampled, attenuation_perturbed, marker="x", color=p.get_facecolor()[0])

    
plt.legend()
plt.xlabel("mu_s")
plt.ylabel("Attenuation")

### Test how many photons need to be stored for good results.

1e5 photons resulted in 44MB of data. Therefore, only try up to 2^21 photons which should result in about 1GB of data.

In [ ]:
for nphoton in [2**j for j in range(10, 22, 2)]:
    print(f"-----------Generating data for {nphoton} photons.------------------")
    cfg["nphoton"] = nphoton
    cfg["maxdetphoton"] = nphoton
    function_data = []
    function_data.append(cfg["nphoton"])
    function_data.append(cfg["prop"][1, 2])
    function_data.append(mu_s_vals)
    for i, mu_s in enumerate(mu_s_vals):
        cfg["prop"][1, 1] = mu_s / 10 # mu_s_vals are stored in cm^-1
        cfg["prop"][1, 0] = 0
        cfg["prop"][1, 2] = g_combined
        cfg["prop"][1, 3] = refractive_index_combined
        res = pmcx.mcxlab(cfg)
        cur_photon_data = np.row_stack(
            (
                res["detp"]["ppath"][:, 0] * cfg["unitinmm"] / 10, # store pathlength in cm. pathlengths are stored in "unitinmm"
                res["detp"]["nscat"][:, 0]
            )
        )
        function_data.append(cur_photon_data)
    fpath = config.mcs_func_path.parent / f"function_data{nphoton}.npz"
    np.savez(fpath, *function_data)

In [ ]:
loader = data_loader.DataLoader(None, 520, 900)
mu_a_matrix = loader.absorption_coefs(
    use_diff_oxycco=False,
    use_water_and_fat=True
)

In [ ]:
spectra_gm, spectra_blood, times = [], [], []
for nphoton in [2**j for j in range(10, 22, 2)]:
    fpath = config.mcs_func_path.parent / f"function_data{nphoton}.npz"
    mcs_obj = SimulationAttenuation(fpath)
    t0 = time.time()
    spectra_gm.append(mcs_obj.A_concentrations(
        loader.wavelengths,
        mu_a_matrix,
        loader.params_ref_gray_matter[:6],
        *loader.params_ref_gray_matter[-2:]
        )
    )

    spectra_blood.append(mcs_obj.A_concentrations(
        loader.wavelengths,
        mu_a_matrix,
        loader.params_ref_blood_vessel[:6],
        *loader.params_ref_blood_vessel[-2:]
    ))
    t1 = time.time()
    times.append((t1 - t0) / 2)
labels = [str(nphoton) for nphoton in [2**i for i in range(10, 22, 2)]]
plot_spectra(spectra_gm, loader.wavelengths, labels, title="Gray Matter")
plot_spectra(spectra_blood, loader.wavelengths, labels, title="Blood")
print(times)

Result: Spectra are almost identical for more than 1e4 photons. Interestingly, 4k photons make very similar spectrum, but 16k photons are noticeably worse. 5e4 photons should be enough. 

In [ ]:
print([t*(24/381) for t in times]) # times for one function evaluation when using 24 wavelengths

### Test how long simulation should run

Simulation should require the longest time (for all backscattered photons to reach the detector) when scattering is at its lowest, because then the pathlength should be its highest. (See e.g. Jacques' model).
Let's compare how the number of captured photons changes when simulation time is reduced.
**Make sure to 'reset' configuration with cell where it is defined!**

In [ ]:
ndetected = []
for simtime in np.logspace(-10, -6, 20):
    cfg["tend"] = simtime
    cfg["tstep"] = simtime
    cfg["prop"][1, 1] = mu_s_vals[-1] / 10 # mu_s_vals are stored in cm^-1
    cfg["prop"][1, 0] = 0
    cfg["prop"][1, 2] = g_combined
    cfg["prop"][1, 3] = refractive_index_combined
    res = pmcx.mcxlab(cfg)
    ndetected.append(res["detp"]["ppath"].shape[0])

In [ ]:
fig = plt.figure()
ax = plt.gca()
ax.scatter(np.logspace(-10, -6, 20), ndetected, label="o")
#ax.set_xscale("log")

A simulation time of 5e7 seconds should be enough for most photons to reach the detector.

Is tissue depth enough? Plot a histogram for the lowest scattering value, and zero absorption.

In [ ]:
function_data_loaded = np.load(config.mcs_func_path)
num_mu_s_vals = len(function_data_loaded["arr_2"])
# remember that pathlengths were stored in cm
# plt.hist(function_data_loaded[f"arr_{num_mu_s_vals - 1}"][0, :] * 10, bins=100, range=[0, 1000])
plt.hist(function_data_loaded[f"arr_{3}"][0, :] * 10, bins=100, range=[0, 1000])

Having a tissue depth of 1000mm seems reasonable. Remember, that photons will typically not travel parallel to z-axis, and pathlength consists of path in and out of tissue. Therefore few photons will reach 1000mm.